In [1]:
#!pip install geopy
#!pip install folium

# The webpage with the postal codes of Toronto Canada will be scrapped to extract the details in a Dataframe using beautiful soup
with algorithm 'lxml'. Relatively most efficient to extract by navigating the tree

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

from collections import Counter
import json
from sklearn.cluster import KMeans
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

column_names = ['PostalCode', 'Borough', 'Neighborhood']
t_rows = 0
t_cols = 0
area = []
borough=[]
neighborhood=[]
postalcode = []

source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

In [3]:
cols = soup.find('td')

In [4]:
postalcode = cols.b.text
print(postalcode)

M1A


In [5]:
span = cols.span.text
print(span)

Not assigned


In [6]:
postalcode=[]
areadetails=[]

for cols in soup.find_all('td'):
    if cols.b != None:
        postalcode.append(cols.b.text)
    if cols.span != None:
        areadetails.append(cols.span.text)

In [7]:
for x in areadetails:
    boroname = ""
    neighdetails = ""
    if (x.find("(") != -1):
        boroname = x[:x.find("(")]
        neighdetails = x[x.find("("):]
        neighdetails = neighdetails.replace("(","")
        neighdetails = neighdetails.replace(")","")
        neighdetails = neighdetails.replace(" / ", ", ")
        borough.append(boroname)
        neighborhood.append(neighdetails)
    else:
        borough.append('Not assigned')
        neighborhood.append('Not assigned')

In [8]:
df = pd.DataFrame([postalcode, borough, neighborhood])
df_Trans = df.T
df_Trans.head(5)

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [9]:
df_Trans.columns = ['Postalcode', 'Borough', 'Neighborhood']

In [10]:
df_Trans.head(5)
df_Trans.drop(df_Trans[df_Trans['Borough']=='Not assigned'].index, inplace=True)

# The dataframe is created with 3 columns, Postalcode, Borough & Neighborhood

In [11]:
toronto_df = df_Trans.reset_index(drop=True)
toronto_df.head(5)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [12]:
toronto_df.shape

(103, 3)

# Using the geospatial dataset, the latitude and longitude coordinates has been extracted from the file for the postal codes 

In [13]:
geo_coordinate_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_coordinate_df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
req_pcode = toronto_df['Postalcode'].to_list()

In [15]:
geo_req_df = geo_coordinate_df[geo_coordinate_df['Postal Code'].isin (req_pcode)]

In [16]:
geo_req_df.sort_values(by='Postal Code', ascending=True)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [17]:
toronto_df_sorted = toronto_df.sort_values(by='Postalcode', ascending=True)

In [18]:
toronto_df_sorted = toronto_df_sorted.set_index('Postalcode').join(geo_req_df.set_index('Postal Code'))

Combined dataframe with postalcode, borough, neighborhood, latitude and longitude has been generated.

In [19]:
toronto_df_sorted.head(5)

,Borough,Neighborhood,Latitude,Longitude
Postalcode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Clustering analysis for the boroughs that contain name Toronto has been initiated

In [20]:
toronto_df_sorted = toronto_df_sorted.reset_index(drop=True)
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df_sorted['Borough'].unique()),
        toronto_df_sorted.shape[0]
    )
)
print(toronto_df_sorted['Borough'].unique())

The dataframe has 15 boroughs and 103 neighborhoods.
['Scarborough' 'North York' 'East York' 'East Toronto'
 'East YorkEast Toronto' 'Central Toronto' 'Downtown Toronto'
 'Downtown TorontoStn A PO Boxes25 The Esplanade' 'York' 'West Toronto'
 "Queen's Park" 'MississaugaCanada Post Gateway Processing Centre'
 'East TorontoBusiness reply mail Processing Centre969 Eastern'
 'Etobicoke' 'EtobicokeNorthwest']


In [21]:
toronto_data = toronto_df_sorted[toronto_df_sorted['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [22]:
#print(toronto_data.shape)
toronto_data.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.676357,-79.293031
1,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
2,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,East Toronto,Studio District,43.659526,-79.340923


In [23]:
from geopy.geocoders import Nominatim 
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tnt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
import folium

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(toronto_data, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

As per the teaching staff comments on discussion forum, please try the below to view the map:
    The maps are not viewable in github.

One way is you can visit https://nbviewer.jupyter.org/ and enter the notebook github link and view it there.
    
Discussion forum link: https://www.coursera.org/learn/applied-data-science-capstone/discussions/weeks/3/threads/ZjrqejTuEeuMagqWRQvY8w

In [25]:
# @hidden_cell
CLIENT_ID = 'XVGF5FBRGJZJULLJEH45AC4H4QHSKFIDFNW4M0U1DFBM55QF' # your Foursquare ID
CLIENT_SECRET = '31MC3RPEJKUKGXTSMZILY140041DAKFMJXFM2O54T4N5T0MG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [26]:
toronto_data.loc[0, 'Neighborhood']

'The Beaches'

In [27]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [28]:
LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60c586a64e16dc11afbfaf47'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 66,
  'suggestedBounds': {'ne': {'lat': 43.685357409000005,
    'lng': -79.28061062898105},
   'sw': {'lat': 43.66735739099998, 'lng': -79.30545177101895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c0e40c0c700c9b6e185a3dd',
       'name': 'Beaches Bake Shop',
       'location': {'address': '900 Kingston Rd.',
        'crossStreet': 'Pickering Ave.',
        'lat': 43.68036312619922,
        'lng': -79.28969173416124,
        'labeledLatLn

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
print(nearby_venues)

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

               venue.name                                   venue.categories  \
0       Beaches Bake Shop  [{'id': '4bf58dd8d48988d16a941735', 'name': 'B...   
1          The Beech Tree  [{'id': '4bf58dd8d48988d155941735', 'name': 'G...   
2         The Fox Theatre  [{'id': '4bf58dd8d48988d17e941735', 'name': 'I...   
3         Tori's Bakeshop  [{'id': '4bf58dd8d48988d1d3941735', 'name': 'V...   
4          Bagels On Fire  [{'id': '4bf58dd8d48988d179941735', 'name': 'B...   
..                    ...                                                ...   
61            Balmy Beach  [{'id': '4bf58dd8d48988d1e2941735', 'name': 'B...   
62         Fiorio Beaches  [{'id': '4bf58dd8d48988d110951735', 'name': 'S...   
63           Running Room  [{'id': '4bf58dd8d48988d1f2941735', 'name': 'S...   
64  Kew Gardens Play Park  [{'id': '4bf58dd8d48988d163941735', 'name': 'P...   
65             Pizzaville  [{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...   

    venue.location.lat  venue.location.

,name,categories,lat,lng
0,Beaches Bake Shop,Bakery,43.680363,-79.289692
1,The Beech Tree,Gastropub,43.680493,-79.288846
2,The Fox Theatre,Indie Movie Theater,43.672801,-79.287272
3,Tori's Bakeshop,Vegetarian / Vegan Restaurant,43.672114,-79.290331
4,Bagels On Fire,Bagel Shop,43.672864,-79.286784


In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head(5)

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Danforth East,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park
4,The Danforth East,43.685347,-79.338106,Danforth & Jones,43.684352,-79.334792,Intersection


In [34]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,47,47,47,47,47,47
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,63,63,63,63,63,63
Christie,14,14,14,14,14,14
Church and Wellesley,65,65,65,65,65,65
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,28,28,28,28,28,28
Davisville North,10,10,10,10,10,10


In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 218 uniques categories.


In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(5)

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_onehot.shape

(1496, 218)

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.042553,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.015873,0.000000,0.000000,0.015873,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.015385,0.015385,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.015385,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.015385
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.035714,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [39]:
toronto_grouped.shape

(39, 218)

In [40]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0                    Coffee Shop  0.09
1                   Cocktail Bar  0.09
2                         Bakery  0.06
3                 Sandwich Place  0.06
4             Seafood Restaurant  0.04
5                 Farmers Market  0.04
6                       Beer Bar  0.04
7  Vegetarian / Vegan Restaurant  0.04
8            Sporting Goods Shop  0.02
9                           Café  0.02


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0          Coffee Shop  0.09
1            Nightclub  0.09
2                 Café  0.09
3       Sandwich Place  0.09
4       Breakfast Spot  0.09
5          Yoga Studio  0.04
6  Japanese Restaurant  0.04
7         Climbing Gym  0.04
8           Restaurant  0.04
9              Stadium  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.12
1 

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Sandwich Place,Bakery,Farmers Market,Beer Bar,Vegetarian / Vegan Restaurant,Seafood Restaurant,Beach,Basketball Stadium
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Sandwich Place,Gym,Bakery,Restaurant,Grocery Store,Japanese Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Boutique,Plane,Rental Car Location,Boat or Ferry,Harbor / Marina,Sculpture Garden,Airport Gate
3,Central Bay Street,Coffee Shop,Sandwich Place,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Café,Restaurant,Pizza Place,Salad Place,Bank
4,Christie,Grocery Store,Café,Park,Baby Store,Nightclub,Restaurant,Italian Restaurant,Coffee Shop,Gas Station,Cuban Restaurant


In [43]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
print(toronto_grouped_clustering)

    Yoga Studio  Adult Boutique  Afghan Restaurant   Airport  \
0      0.000000        0.000000           0.000000  0.000000   
1      0.043478        0.000000           0.000000  0.000000   
2      0.000000        0.000000           0.000000  0.058824   
3      0.015873        0.000000           0.000000  0.000000   
4      0.000000        0.000000           0.000000  0.000000   
5      0.015385        0.015385           0.015385  0.000000   
6      0.000000        0.000000           0.000000  0.000000   
7      0.000000        0.000000           0.000000  0.000000   
8      0.000000        0.000000           0.000000  0.000000   
9      0.000000        0.000000           0.000000  0.000000   
10     0.058824        0.000000           0.000000  0.000000   
11     0.000000        0.000000           0.000000  0.000000   
12     0.000000        0.000000           0.000000  0.000000   
13     0.000000        0.000000           0.000000  0.000000   
14     0.000000        0.000000         

In [44]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 5, 2, 1,
       2, 2, 2, 2, 3, 4, 2, 2])

In [45]:
# add clustering labels
neighborhoods_venues_sorted.head(5)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Pub,Wine Shop,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,3,Park,Convenience Store,Intersection,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
2,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pub,Bank,Indian Restaurant,Sandwich Place,Juice Bar
3,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Food & Drink Shop,Board Shop,Liquor Store,Restaurant,Brewery,Italian Restaurant,Pub,Burrito Place,Ice Cream Shop
4,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Gastropub,Café,Bakery,Furniture / Home Store,Stationery Store,Cheese Shop,Seafood Restaurant,Italian Restaurant,Fish Market


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

As per the teaching staff comments on discussion forum, please try the below to view the map:
    The maps are not viewable in github.

One way is you can visit https://nbviewer.jupyter.org/ and enter the notebook github link and view it there.
    
Discussion forum link: https://www.coursera.org/learn/applied-data-science-capstone/discussions/weeks/3/threads/ZjrqejTuEeuMagqWRQvY8w

Observation: The clusters are formed indicate that the neighborhoods are grouped by 
    1. Cluster #1 - Mostly clustered based on gathering and eatery places.
    2. Cluster #2 - Not a morning rush location
    3. Cluster #3 - More like a suburb location
    4. Cluster #4 - Main attractions include garden and wineshop
    5. Cluster #5 - Categorized for other services, home and distribution
    6. Cluster #6 - Is categorized as a transit location for activities

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Health Food Store,Pub,Wine Shop,Cupcake Shop,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
3,"India Bazaar, The Beaches West",Fast Food Restaurant,Food & Drink Shop,Board Shop,Liquor Store,Restaurant,Brewery,Italian Restaurant,Pub,Burrito Place,Ice Cream Shop
31,Christie,Grocery Store,Café,Park,Baby Store,Nightclub,Restaurant,Italian Restaurant,Coffee Shop,Gas Station,Cuban Restaurant
32,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Smoke Shop,Café,Bank,Bar,Middle Eastern Restaurant,Pet Store,Supermarket,Brewery
35,"High Park, The Junction South",Bar,Mexican Restaurant,Café,Grocery Store,Thai Restaurant,Fast Food Restaurant,Diner,Speakeasy,Italian Restaurant,Bakery
38,Enclave of M4L,Light Rail Station,Gym / Fitness Center,Smoke Shop,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Moore Park, Summerhill East",Trail,Wine Shop,Dance Studio,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Yoga Studio,Pub,Bank,Indian Restaurant,Sandwich Place,Juice Bar
4,Studio District,Coffee Shop,Gastropub,Café,Bakery,Furniture / Home Store,Stationery Store,Cheese Shop,Seafood Restaurant,Italian Restaurant,Fish Market
6,Davisville North,Gym / Fitness Center,Food & Drink Shop,Hotel,Playground,Department Store,Pizza Place,Breakfast Spot,Sandwich Place,Park,Gym
7,North Toronto West,Coffee Shop,Clothing Store,Gym / Fitness Center,Bagel Shop,Health & Beauty Service,Fast Food Restaurant,Diner,Mexican Restaurant,Miscellaneous Shop,Park
8,Davisville,Dessert Shop,Sushi Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Optical Shop,Park,Pharmacy,Café,Brewery
10,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Light Rail Station,Pizza Place,Vietnamese Restaurant,Sushi Restaurant,Bagel Shop,Bank,American Restaurant,Dessert Shop,Diner
12,"St. James Town, Cabbagetown",Pizza Place,Coffee Shop,Café,Bakery,Restaurant,Pub,Pharmacy,Chinese Restaurant,Italian Restaurant,Diner
13,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Men's Store,Mediterranean Restaurant,Indian Restaurant,Gym,Gay Bar,Fast Food Restaurant
14,"Regent Park, Harbourfront",Coffee Shop,Bakery,Park,Pub,Café,Beer Store,Bank,Spa,Performing Arts Venue,Farmers Market
15,"Garden District, Ryerson",Coffee Shop,Sandwich Place,Clothing Store,Café,Hotel,Cosmetics Shop,Bank,Pizza Place,Japanese Restaurant,Thai Restaurant


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,The Danforth East,Park,Convenience Store,Intersection,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
11,Rosedale,Park,Trail,Playground,Wine Shop,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store


In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Roselawn,Home Service,Music Venue,Garden,Wine Shop,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Lawrence Park,Park,Bus Line,Swim School,College Gym,Dance Studio,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
